# Data-Import

In [448]:
import pandas as pd
import ast
import re

# Set the display options to show all columns and rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# Data import Aisha
original_dataframe = pd.read_csv("recipes_w_search_terms.csv")

# data import katie
# original_dataframe = pd.read_csv(
#    "C:\\Users\\k.ritscher\\Desktop\\Bootcamp\\Final_Project\\archive\\recipes_w_search_terms.csv"
# )

### Dropping Duplicates, Creating smaller DF

In [449]:
# drop duplicates
df = original_dataframe.copy()
df.drop_duplicates(
    subset=["name", "ingredients", "description"], keep="first", inplace=True
)

In [450]:
# see difference in the size of the df,
print(len(original_dataframe))
print(len(df))

494963
494777


In [451]:
# creating a smaller test-dataframe
test_df = df.iloc[0:30, :]

## Creating columns 

-  basic_ingredients
-  non_basic_ingredients
-  ingredients
-  time to make
-  low-in-something
-  free-off-something
-  cuisine
-  main ingredients
-  meal time

### Create column for "basic_ingredients"

In [452]:
df["ingredients"].dtypes  # the datatype of ingredients seems to be an object

dtype('O')

In [453]:
# New column for Basic ingredients

# Create a loop to check for ingredients in ingredients column
ingredient_check = [
    ingredient
    for ingredient in [
        "salt",
        "oil",
        "water",
        "pepper",
        "vegetable oil",
        "garlic",
        "olive oil",
        "white wine vinegar",
        "table salt",
        "black pepper",
        "pepper and salt",
        "salt and pepper",
        "italian seasoning",
        "flour",
        "allspice",
        "cinnamon",
        "sugar",
        "ground cumin",
        "cumin",
        "red bell pepper",
        "cayenne pepper",
        "nutmeg",
        "baking powder",
        "all purpose flour",
        "white vinegar",
        "ice",
        "cardamom",
        "kosher salt",
        "oregano",
    ]
    if any(
        ingredient in row
        for row in test_df["ingredients"]
        .str.strip("[]")
        .str.replace("'", "")
        .str.split(", ")
    )
]

# Replace " ' " and separate by ,
processed_ingredients = [
    ingredient_list.replace("'", "").split(", ")
    for ingredient_list in test_df["ingredients"].to_list()
]

# Create new column with basic ingredients
basic_ingredients_list = [
    "salt",
    "oil",
    "water",
    "pepper",
    "vegetable oil",
    "garlic",
    "olive oil",
    "white wine vinegar",
    "table salt",
    "black pepper",
    "pepper and salt",
    "salt and pepper",
    "italian seasoning",
    "flour",
    "allspice",
    "cinnamon",
    "sugar",
    "ground cumin",
    "cumin",
    "red bell pepper",
    "cayenne pepper",
    "nutmeg",
    "baking powder",
    "all purpose flour",
    "white vinegar",
    "ice",
    "cardamom",
    "kosher salt",
    "oregano",
]
# create column called basic_ingredients by looping ingredients through just created list, strip etc for format
test_df["basic_ingredients"] = [
    [ingredient for ingredient in basic_ingredients_list if ingredient in row]
    for row in test_df["ingredients"]
    .str.strip("[]")
    .str.replace("'", "")
    .str.split(", ")
]

C:\Users\k.ritscher\AppData\Local\Temp\ipykernel_14352\909591390.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["basic_ingredients"] = [


### Create column for "non_basic_ingredients"
goal: wanting to create a column that show all ingredients from ingredient column that dont already appear in basic ingredients
those ingredients will be the ones that you write in the app, the ones that you have left over in the fridge

In [454]:
# write function for dealing with format
def extract_list_from_col(column_item: str) -> list:
    return column_item.strip("[]").replace("'", "").split(", ")



# write function for looping through ingredients and make sure they are not in basic_ingredients_column


def extract_non_basic(ingredients, basic_ingredients):
    return [
        ingredient

        for ingredient in extract_list_from_col(ingredients)
        if not ingredient in basic_ingredients

    ]



# itterare trhough dataframe for each row, by applying the function that we created above



for index, row in test_df.iterrows():
    print(extract_non_basic(row["ingredients"], row["basic_ingredients"]))

    break

# create column
test_df = test_df.assign(  # assigning means creating a new column or change an existing one
    non_basic_ingredients=[
        extract_non_basic(
            row["ingredients"], row["basic_ingredients"]
        )  # calling function from above
        for _, row in test_df.iterrows()  # iterrows works with index and data,this is why we have 2 variables after "for"
    ]
)
# if I want to learn more: youtube - list comprehension python

['grits', 'cheddar cheese']


### Create column for "time to make"

In [456]:
# Function to extract 'time-to-make' value based on 'minutes' or 'hours'
def extract_time_to_make(tags_list):
    time_to_make_values = []
    for tag in tags_list:
        if "minutes" in tag.lower() or "hours" in tag.lower():
            time_to_make_values.append(tag.strip())
    return time_to_make_values if time_to_make_values else None


# Apply the function to create the new 'time-to-make' column
test_df["time_to_make"] = df["tags"].apply(
    lambda x: extract_time_to_make(ast.literal_eval(x)) if pd.notnull(x) else None
)

### Create a column for low_in_something

In [458]:
import re


# Function to extract 'low-in-something' value based on 'low'
def extract_low_in_something(low_values_list):
    low_in_something_values = []
    for tag in low_values_list:
        # Check if the tag has 'low' as a whole word
        if re.search(r"\blow\b", tag.lower()):
            low_in_something_values.append(tag.strip())
    return low_in_something_values if low_in_something_values else None


# Apply the function to create the new 'low-in-something' column
test_df["low_in_something"] = test_df["tags"].apply(
    lambda x: extract_low_in_something(ast.literal_eval(x)) if pd.notnull(x) else None
)

# # converting column from list to string
# test_df["non_basic_ingredients"] = [
#     ",".join(map(str, l)) for l in test_df["non_basic_ingredients"]
# ]
# test_df.head(1)

In [459]:
# # converting column from list to string
# test_df["low-in-something"] = [
#     ",".join(map(str, l)) for l in test_df["low-in-something"]
# ]
# test_df["low-in-something"].dtypes

### Add a column for "free_of_something"

In [460]:
import re


# Function to extract 'free-of-something' value based on 'free'
def extract_free_of_something(free_values_list):
    free_of_something_values = []
    for tag in free_values_list:
        # Check if the tag has 'free' as a whole word
        if re.search(r"\bfree\b", tag.lower()):
            free_of_something_values.append(tag.strip())
    return free_of_something_values if free_of_something_values else None


# Apply the function to create the new 'free-of-something' column
test_df["free_of_something"] = test_df["tags"].apply(
    lambda x: extract_free_of_something(ast.literal_eval(x)) if pd.notnull(x) else None
)

In [461]:
# # converting column from list to string
# test_df["free_of_something"] = (
#     pd.DataFrame(test_df.free_of_something.tolist())
#     .fillna("")
#     .astype(str)
#     .agg(",".join, 1)
#     .str.strip(",")
# )

### Add a column for cuisines 

In [462]:
# List of predefined cuisines
cuisines = [
    "indian",
    "american",
    "lebanese",
    "italian",
    "chinese",
    "mexican",
    "japanese",
    "french",
    "german",
    "greek",
    "spanish",
    "thai",
    "korean",
    "vietnamese",
    "brazilian",
    "moroccan",
    "turkish",
    "russian",
    "swedish",
    "caribbean",
    "ethiopian",
    "peruvian",
    "malaysian",
    "australian",
    "south african",
    "portuguese",
    "cajun/creole",
    "argentine",
    "filipino",
    "middle-eastern",
    "north-american",
    "central-american",
    "south-american",
    "european",
    "eastern-european",
    "western-european",
    "southeast-asian",
    "south-asian",
    "middle-eastern",
    "north-african",
    "sub-saharan-african",
    "south-indian",
    "north-indian",
    "southern-american",
    "northern-american",
    "central-european",
    "northern-european",
    "southern-european",
    "eastern-asian",
    "western-asian",
    "northern-african",
    "southern-african",
    "east-african",
    "west-african",
    "eastern-european",
    "western-european",
    "southeast-asian",
    "south-asian",
    "east-asian",
    "central-asian",
    "western-asian",
    "middle-eastern",
    "south-east-asian",
    "north-east-asian",
    "south-central-asian",
    "north-central-asian",
    "south-west-asian",
    "north-west-asian",
    "south-east-european",
    "north-east-european",
    "south-central-european",
    "north-central-european",
    "south-west-european",
    "north-west-european",
    "south-east-african",
    "north-east-african",
    "south-central-african",
    "north-central-african",
    "south-west-african",
    "north-west-african",
    "southwest",
    "southwestern-us",
    "southwestern-american",
    "southwestern",
    "south-west-pacific",
]


# Function to extract 'cuisine' value
def extract_cuisines(tags_list):
    cuisine_list = [tag.strip() for tag in tags_list if tag.lower() in cuisines]
    return cuisine_list if cuisine_list else None


# Apply the function to create the new 'cuisine' column and replace NaN with None
test_df["cuisine"] = (
    test_df["tags"]
    .apply(lambda x: extract_cuisines(ast.literal_eval(x)) if pd.notnull(x) else None)
    .replace({pd.NA: None})
)

test_df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,non_basic_ingredients,time_to_make,low_in_something,free_of_something,cuisine
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}","[salt, water, garlic, olive oil]","[grits, cheddar cheese]",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None,None


### Add a column for Main Ingredients

In [487]:
import ast
import re


# Function to apply to each row
def find_main_ingredients(row):
    # Recipe name (converted to lowercase)
    recipe_name = row["name"].lower()

    # Convert the string representation of the list to an actual list of strings
    ingredients_list = ast.literal_eval(row["ingredients"])

    # Extract words from ingredients and convert to lowercase
    ingredients_lower = [
        word.lower().strip()
        for ingredient in ingredients_list
        for word in re.findall(r"\b[\w\s-]+\b", ingredient)
    ]

    # Find ingredients mentioned in the recipe name
    main_ingredients = []

    for ingredient in ingredients_lower:
        if ingredient in recipe_name or any(
            part in recipe_name for part in ingredient.split()
        ):
            main_ingredients.append(ingredient)

    return main_ingredients


# Apply the function to each row in 'test_df'
test_df["main_ingredients"] = test_df.apply(find_main_ingredients, axis=1)

# Display the resulting DataFrame
test_df.head(1)

### Create column for "sweet or savoury"

In [464]:
# Function to extract 'sweet' or 'savoury' value and make a new column
def extract_sweet_or_savoury(tags_list):
    sweet_or_savoury_values = []
    for tag in tags_list:
        if "sweet" in tag.lower() or "dessert" in tag.lower():
            sweet_or_savoury_values.append(tag.strip())
    return sweet_or_savoury_values if sweet_or_savoury_values else "Savoury"


# Apply the function to create the new 'time-to-make' column
test_df["sweet_or_savoury"] = test_df["tags"].apply(
    lambda x: extract_sweet_or_savoury(ast.literal_eval(x))
    if pd.notnull(x)
    else "Savoury"
)

### Create a column for meal time

In [465]:
import ast


# Function to extract 'meal time' (Breakfast, lunch, or dinner)
def extract_time_to_make(tags_list, search_terms_list):
    meal_time_values = []

    for tag in tags_list:
        if (
            "breakfast" in tag.lower()
            or "lunch" in tag.lower()
            or "dinner" in tag.lower()
            or "brunch" in tag.lower()
            or "supper" in tag.lower()
        ):
            meal_time_values.append(tag.strip())

    for term in search_terms_list:
        if (
            "breakfast" in term.lower()
            or "lunch" in term.lower()
            or "dinner" in term.lower()
            or "brunch" in term.lower()
            or "supper" in tag.lower()
        ):
            meal_time_values.append(term.strip())

    return meal_time_values if meal_time_values else "Appetizer/Sides/Snakcs"


# Apply the function to create the new 'meal time' column
test_df["meal_time"] = test_df.apply(
    lambda row: extract_time_to_make(
        ast.literal_eval(row["tags"]), ast.literal_eval(row["search_terms"])
    )
    if pd.notnull(row["tags"]) or pd.notnull(row["search_terms"])
    else "Appetizer/Sides/Snakcs",
    axis=1,
)

# Display the resulting DataFrame
test_df.head(2)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,non_basic_ingredients,time_to_make,low_in_something,free_of_something,cuisine,sweet_or_savoury,meal_time
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}","[salt, water, garlic, olive oil]","[grits, cheddar cheese]",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None,None,Savoury,[brunch]
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are in the mood for jambalaya. My kids loved it! From Cook's Illustrated ""Quick Recipe"" Cookbook.","['onion', 'red bell pepper', 'garlic cloves', 'large shrimp', 'salt', 'hot pepper sauce', 'vegetable oil', 'andouille sausage', 'long grain rice', 'bay leaves', 'diced tomatoes', 'clam juice', 'fresh parsley']","[""1 medium onion, chopped coarse "",""1 medium red bell pepper, chopped coarse "",""5 medium garlic cloves, chopped coarse "",""1 lb extra large shrimp, shelled and deveined "","" salt"","" hot pepper sauce"",""1 tablespoon vegetable oil"",""3/4 lb andouille sausage, halved lengthwise and then cut into 1/4 inch slices "",""1 1/2 cups long grain rice"",""4 bay leaves"",""1 (14 ounce) can diced tomatoes, briefly drained "",""2 (8 ounce) bottles clam juice"",""1/4 cup fresh parsley, chopped ""]",1 (366 g),4,"['In a food processor, pulse the onion, red pepper and garlic until chopped very fine, ten to twelve 1-second pulses, scraping down the sides of the bowl as necessary; set aside.', 'Season the shrimp with salt and hot pepper sauce to taste.', 'Heat the oil in a large Dutch oven over medium high heat until shimmering. Add the shrimp in a single layer and cook, without stirring, for 30 seconds. Using tongs, flip the shrimp and cook for another 30 seconds. Transfer the shrimp to a medium bowl and set aside.', 'Add the sausage to the pan and cook, stirring occasionally, until lightly browned, about 3 minutes. Using a slotted spoon, transfer the sausage to a second bowl.', 'Scrape the veggies from the food processor into the empty pot and cook, stirring frequently, until softened, about 3 minutes. Stir in the rice and continue to stir until the grains are coated with the fat, about 1 minute.', 'Add the bay leaves, tomatoes, clam juice, and 1 cup water and bring to a boil.', 'Stir in the sausage, cover, and reduce the heat to low.', 'Cook until the ric

In [466]:
# Create a new csv file with test_df
test_df.to_csv("./test_df.csv", index=False)

## Converting columns 

#### converting columns of original dataframe

In [467]:
# converting column from list to string
test_df["steps"] = test_df["steps"].apply(lambda x: ", ".join(eval(x)))
test_df["tags"] = test_df["tags"].apply(lambda x: ", ".join(eval(x)))
test_df["search_terms"] = test_df["search_terms"].apply(lambda x: ", ".join(eval(x)))
test_df["ingredients"] = test_df["ingredients"].apply(lambda x: ", ".join(eval(x)))

#### converting basic and non basic ingredient columns

In [468]:
# converting column from list to string
test_df["basic_ingredients"] = [
    ",".join(map(str, l)) for l in test_df["basic_ingredients"]
]

# converting column from list to string
test_df["non_basic_ingredients"] = [
    ",".join(map(str, l)) for l in test_df["non_basic_ingredients"]
]

#### converting other columns

In [469]:
# converting column from list to string
test_df["time_to_make"] = test_df["time_to_make"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
test_df["low_in_something"] = test_df["low_in_something"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
test_df["free_of_something"] = test_df["free_of_something"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
test_df["cuisine"] = test_df["cuisine"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
test_df["sweet_or_savoury"] = test_df["sweet_or_savoury"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
test_df["meal_time"] = test_df["meal_time"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
test_df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,basic_ingredients,non_basic_ingredients,time_to_make,low_in_something,free_of_something,cuisine,sweet_or_savoury,meal_time
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","water, grits, salt, cheddar cheese, garlic, olive oil","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally., Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim., Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil., Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.","time-to-make, course, main-ingredient, preparation, occasion, side-dishes, eggs-dairy, refrigerator, diabetic, vegetarian, grains, cheese, stove-top, dietary, low-cholesterol, low-calorie, comfort-food, low-carb, low-in-something, pasta-rice-and-grains, brunch, taste-mood, equipment, presentation, served-hot, 4-hours-or-less","low-calorie, side, low-carb, diabetic, vegetarian","salt,water,garlic,olive oil","grits,cheddar cheese",4-hours-or-less,"low-cholesterol, low-calorie, low-carb, low-in-something",None,None,Savoury,brunch


# Unused Code

In [470]:
# def filter_basic_ingredients(ingredient_list):
#     basic_ingredients = [
#         "salt",
#         "oil",
#         "water",
#     ]

#     return [
#         ingredient
#         for ingredient in basic_ingredients
#         if ingredient.lower() in ingredient_list
#     ]

In [471]:
# # Function to ensure that each cell is a list
# def ensure_list(cell):
#     if isinstance(cell, list):
#         return cell
#     else:
#         return [cell]


# # Apply the function to the "ingredients" column
# test_df["ingredients"] = test_df["ingredients"].apply(ensure_list)

# test_df.head(1)

In [472]:
# filter_basic_ingredients(test_df["ingredients"])

In [473]:
# import numpy as np

# def filter_basic_ingredients(ingredient_list):
#     return [
#         ingredient
#         for ingredient in ingredient_list
#         if ingredient.lower() in basic_ingredients
#     ]

#     return [
#         ingredient
#         for ingredient in ingredient_list
#         if ingredient.lower() in basic_ingredients
#     ]

# # test_df["filtered_ingredients"] = np.where(
# #     df["ingredients"].isin(basic_ingredients), "Yes", "No"
# # )

# test_df["filtered_ingredients"] = np.where(
#     test_df["ingredients"].isin(basic_ingredients),
#     (test_df["ingredients"].apply(filter_basic_ingredients())),
# )

In [474]:
# def filter_basic_ingredients(ingredient_list):
#     basic_ingredients = [
#         "salt",
#         "oil",
#         "water",
#     ]  # Removed the extra double quote in 'salt'

#     return [
#         ingredient
#         for ingredient in ingredient_list
#         if ingredient.lower() in basic_ingredients
#     ]  # Use lower() to perform case-insensitive comparison


# test_df["filtered_ingredients"] = test_df[["ingredients"][0]].apply(
#     filter_basic_ingredients
# )

# # Display the modified DataFrame
# test_df.head(1)

In [475]:
# def filter_basic_ingredients(ingredient_list):
#     basic_ingredients = ["salt", "oil", "water"]

#     filtered_list = []
#     for sublist in ingredient_list:
#         for ingredient in sublist:
#             if ingredient.lower() in basic_ingredients:
#                 filtered_list.append(ingredient.lower())

#     return filtered_list

In [476]:
# test_df["filtered_ingredients"] = test_df["ingredients"].apply(
# filter_basic_ingredients()
# )

# test_df["filtered_ingredients"] = np.where(
#     df["ingredients"].isin(basic_ingredients), "Yes", "No"
# )

# test_df["filtered_ingredients"] = filter_basic_ingredients(test_df["ingredients"])


# Display the modified DataFrame
# test_df.head(1)

# look into  apply method, stack overflow
# otherwise, find an alternative (copy column, extract ingredients, send to yanish)

In [477]:
# # New column for Basic ingredients

# # Create a loop to check for ingredients in ingredients column
# ingredient_check = [
#     ingredient
#     for ingredient in ["salt", "oil", "water", "pepper", "vegetable oil"]
#     if any(ingredient in row for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", "))
# ]

# # Replace " ' " and separate by ,
# processed_ingredients = [
#     ingredient_list.replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]

# # Create new column with basic ingredientsThird block
# basic_ingredients_list = ["salt", "oil", "water", "pepper", "vegetable oil"]
# test_df["basic_ingredients"] = [
#     [ingredient for ingredient in basic_ingredients_list if ingredient in row]
#     for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", ")
# ]

# test_df.head(2)

In [478]:
# test_df["basic_ingredients"] = [
#     ingredient
#     for ingredient in [
#         "salt",
#         "oil",
#         "water",
#     ]
#     if ingredient
#     in test_df["ingredients"][1]
#     .strip("][]")
#     .replace("'", "")
#     .split(
#         ", "
#     )  # transform column to a list by removing squarebrackets and quotes and split by commas
# ]

# test_df.head(1)

In [479]:
# # New column for Basic ingredients

# # Create a loop to check for ingredients in ingredients column
# ingredient_check = [
#     ingredient
#     for ingredient in ["salt", "oil", "water", "pepper", "vegetable oil"]
#     if any(ingredient in row for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", "))
# ]

# # Replace " ' " and separate by ,
# processed_ingredients = [
#     ingredient_list.replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]

# # Create new column with basic ingredientsThird block
# basic_ingredients_list = ["salt", "oil", "water", "pepper", "vegetable oil"]
# test_df["basic_ingredients"] = [
#     [ingredient for ingredient in basic_ingredients_list if ingredient in row]
#     for row in test_df["ingredients"].str.strip("[]").str.replace("'", "").str.split(", ")
# ]

# test_df.head(2)

In [480]:
# [
#     ingredient_list.strip("][").replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]

# test_df["non_basic_ingredients"] = ~(
#     test_df["ingredients"].isin(test_df["basic_ingredients"])
# )

In [481]:
# def non_basic_ingredients_function(row):
#     # Check the type of data in the columns
#     # print(type(row["ingredients"]), type(row["basic_ingredients"]))

#     set_a = set(row["ingredients"])
#     set_b = set(row["basic_ingredients"])
#     missing_values = list(set_a - set_b)
#     return missing_values


# # Apply the function to create the new 'non_basic_ingredients' column
# test_df["non_basic_ingredients"] = test_df.apply(non_basic_ingredients_function, axis=1)

# test_df.head(2)

In [482]:
# def extract_non_basic(ingredient_extract_list):
#     list_of_ingredients = []
#     for x in ingredient_extract_list:
#         if x not in test_df["basic_ingredients"].tolist():
#             cleaned_ingredient = x.strip("[]").replace("'", "")
#             list_of_ingredients.extend(cleaned_ingredient.split(", "))
#     return list_of_ingredients


# test_df["non_basic_ingredients"] = test_df["ingredients"].apply(extract_non_basic)

# test_df.head(1)

In [483]:
# understanding creating column for non-basic-ingredients

# non_basic_ingredientss = []
# for _, row in test_df.iterrows():
#     non_basic_ingredients_list = extract_non_basic(row["ingredients"], row["basic_ingredients"])
#     non_basic_ingredientss.append(non_basic_ingredients_list)
# test_df = test_df.assign(non_basic_ingredients=non_basic_ingredientss)

# if I want to learn more: youtube - list comprehension python

#### Tims Code

In [484]:
##Tim
# [
#     ingredient
#     for ingredient in [
#         "salt",
#         "oil",
#         "water",
#     ]
#     if ingredient
#     in test_df["ingredients"][0]
#     .strip("][]")
#     .replace("'", "")
#     .split(
#         ", "
#     )  # transform column to a list by removing squarebrackets and quotes and split by commas
# ]
# # next: loop through ingredients again

In [485]:
# [
#     ingredient_list.strip("][").replace("'", "").split(", ")
#     for ingredient_list in test_df["ingredients"].to_list()
# ]

In [486]:
# understanding creating column for non-basic-ingredients

# non_basic_ingredientss = []
# for _, row in test_df.iterrows():
#     non_basic_ingredients_list = extract_non_basic(row["ingredients"], row["basic_ingredients"])
#     non_basic_ingredientss.append(non_basic_ingredients_list)
# test_df = test_df.assign(non_basic_ingredients=non_basic_ingredientss)

# if I want to learn more: youtube - list comprehension python